In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('dados/dataset_with_location_filters.csv').drop(['Unnamed: 0','EASE-MENT'], axis=1)

cond_1 = (df["YEAR BUILT"] == 0)
cond_2 = (df["GROSS SQUARE FEET"] == ' -  ')
cond_3 = (df["GROSS SQUARE FEET"] == '0')
cond_4 = (df["LAND SQUARE FEET"] == '0')
cond_5 = (df["LAND SQUARE FEET"] == ' -  ')
cond_6 = (df["ZIP CODE"] == 0)
new_df = df[~(cond_1 | cond_2 | cond_3 | cond_4 | cond_5 | cond_6)]

print(new_df.shape)
new_df.head()

(28246, 24)


,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,...,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,sale_month,sale_week,sale_year,latitude,longitude,inside
0,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,C2,,10009,5,...,1900,2,C2,6625000.0,7,29,2017,40.726568,-73.979858,True
1,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,C4,,10009,10,...,1913,2,C4,3936272.0,9,38,2016,40.725445,-73.982434,True
2,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,C2,,10009,6,...,1900,2,C2,8000000.0,11,46,2016,40.727795,-73.981660,True
3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,406,32,C4,,10009,8,...,1920,2,C4,3192840.0,9,38,2016,40.728595,-73.978855,True
4,1,ALPHABET CITY,08 RENTALS - ELEVATOR APARTMENTS,2,387,153,D9,,10009,24,...,1920,2,D9,16232000.0,11,45,2016,40.723564,-73.980730,True


In [3]:
new_df['GROSS SQUARE FEET'] = new_df['GROSS SQUARE FEET'].astype(int)
new_df['LAND SQUARE FEET'] = new_df['LAND SQUARE FEET'].astype(int)

new_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28246 entries, 0 to 28245
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         28246 non-null  int64  
 1   NEIGHBORHOOD                    28246 non-null  object 
 2   BUILDING CLASS CATEGORY         28246 non-null  object 
 3   TAX CLASS AT PRESENT            28246 non-null  object 
 4   BLOCK                           28246 non-null  int64  
 5   LOT                             28246 non-null  int64  
 6   BUILDING CLASS AT PRESENT       28246 non-null  object 
 7   APARTMENT NUMBER                28246 non-null  object 
 8   ZIP CODE                        28246 non-null  int64  
 9   RESIDENTIAL UNITS               28246 non-null  int64  
 10  COMMERCIAL UNITS                28246 non-null  int64  
 11  TOTAL UNITS                     28246 non-null  int64  
 12  LAND SQUARE FEET                

In [4]:
#tipos das variáveis
var_cat = ['BOROUGH','NEIGHBORHOOD','BUILDING CLASS CATEGORY','TAX CLASS AT PRESENT','BLOCK','LOT' ,'BUILDING CLASS AT PRESENT',
           'APARTMENT NUMBER', 'ZIP CODE','TAX CLASS AT TIME OF SALE','BUILDING CLASS AT TIME OF SALE']

var_num = ['RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS', 'LAND SQUARE FEET', 'GROSS SQUARE FEET','YEAR BUILT',
           'sale_month','sale_week','sale_year', 'latitude','longitude']

#### Pré-processamento

In [5]:
y = new_df['SALE PRICE']
X = new_df[var_num]#.drop('SALE PRICE',axis=1)

X_train, X_test, y_train, y_test = train_test_split(X,np.array(y).reshape(-1,1), test_size=0.2, random_state=42)

scaler = MinMaxScaler().fit(X_train)
yscaler = MinMaxScaler().fit(y_train)

treino = scaler.transform(X_train)
teste = scaler.transform(X_test)
y_treino = yscaler.transform(y_train)
y_teste = yscaler.transform(y_test)

#### Clusterização

In [6]:
#kmeans sem tunning
k=5
kmeans = KMeans(n_clusters=k, random_state=42).fit(treino)
new_treino = treino
new_teste = teste
for i in range(k):
    new_treino =np.c_[new_treino,np.linalg.norm(treino-kmeans.cluster_centers_[i], axis=1)]
    new_teste =np.c_[new_teste,np.linalg.norm(teste-kmeans.cluster_centers_[i], axis=1)]


# atualizando a lista de colunas
var_num_mod = var_num
for i in range(k):
    var_num_mod.append('deucli_k' + str(i+1))

# bases de treino e teste com as novas features
new_features_train = pd.DataFrame(np.c_[new_treino,y_treino,new_df.loc[X_train.index, var_cat]],
                                  columns=var_num_mod + ['SALE PRICE'] + var_cat)
new_features_train.to_csv('dados/treino+kmeans.csv')
new_features_test = pd.DataFrame(np.c_[new_teste,y_teste,new_df.loc[X_test.index, var_cat]],
                                  columns=var_num_mod + ['SALE PRICE'] + var_cat)
new_features_test.to_csv('dados/teste+kmeans.csv')

#Shape das novas bases
print(new_features_train.shape)
print(new_features_test.shape)
new_features_train.head()

(22596, 28)
(5650, 28)


,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,sale_month,sale_week,sale_year,latitude,...,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,APARTMENT NUMBER,ZIP CODE,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE
0,0.0010846,0,0.000884564,0.000469478,0.000535941,0.599078,0.727273,0.745098,0,0.970572,...,WAKEFIELD,02 TWO FAMILY DWELLINGS,1,5094,19,B2,,10466,1,B2
1,0.0016269,0,0.00132685,0.000662236,0.0006026,0.529954,0.727273,0.666667,0,0.280116,...,BENSONHURST,03 THREE FAMILY DWELLINGS,1,6162,20,C0,,11204,1,C0
2,0.000542299,0,0.000442282,0.000491947,0.000241573,0.806452,0.0909091,0.117647,1,0.219852,...,NEW SPRINGVILLE,01 ONE FAMILY DWELLINGS,1,2379,62,A5,,10314,1,A5
3,0.0010846,0,0.000884564,0.000639294,0.000898567,0.506912,0.181818,0.235294,1,0.8574,...,FORDHAM,02 TWO FAMILY DWELLINGS,1,3144,31,B1,,10457,1,B1
4,0.000542299,0,0.000442282,0.00060429,0.000263971,0.668203,0.0909091,0.0980392,1,0.431155,...,SPRINGFIELD GARDENS,01 ONE FAMILY DWELLINGS,1,12270,103,A2,,11434,1,A2


In [7]:
pd.DataFrame(new_treino, columns=var_num)

,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,sale_month,sale_week,sale_year,latitude,longitude,deucli_k1,deucli_k2,deucli_k3,deucli_k4,deucli_k5
0,0.001085,0.0,0.000885,0.000469,0.000536,0.599078,0.727273,0.745098,0.0,0.970572,0.736285,0.430798,1.147201,0.833707,1.482959,1.374204
1,0.001627,0.0,0.001327,0.000662,0.000603,0.529954,0.727273,0.666667,0.0,0.280116,0.478430,0.457264,1.094261,0.298836,1.189500,1.316088
2,0.000542,0.0,0.000442,0.000492,0.000242,0.806452,0.090909,0.117647,1.0,0.219852,0.178294,1.607969,0.835482,1.467592,0.298422,0.634867
3,0.001085,0.0,0.000885,0.000639,0.000899,0.506912,0.181818,0.235294,1.0,0.857400,0.642640,1.392799,0.538119,1.518168,0.802331,0.357094
4,0.000542,0.0,0.000442,0.000604,0.000264,0.668203,0.090909,0.098039,1.0,0.431155,0.858368,1.482634,0.600207,1.555691,0.713024,0.213486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22591,0.000542,0.0,0.000442,0.000426,0.000289,0.552995,0.454545,0.431373,1.0,0.445510,0.792572,1.169397,0.163795,1.255123,0.650089,0.448043
22592,0.001085,0.0,0.000885,0.000367,0.000240,0.622120,0.727273,0.745098,0.0,0.287869,0.464941,0.424130,1.106711,0.210037,1.199709,1.345914
22593,0.000542,0.0,0.000442,0.000204,0.001312,0.552995,0.636364,0.666667,1.0,0.708480,0.504650,1.075212,0.348776,1.152952,0.766016,0.762944
22594,0.001085,0.0,0.000885,0.000411,0.000304,0.552995,0.818182,0.764706,0.0,0.653139,0.763345,0.147889,1.096738,0.593901,1.402645,1.363983


Um coeficiente de silhueta de 0 indica que os clusters estão se sobrepondo significativamente, e um coeficiente de silhueta de 1 indica que os clusters estão bem separados.

In [ ]:
silhouette_score(treino, kmeans.labels_).round(2)